configuração e inicialização do SparkSession

In [1]:
!pip install pyspark #Instala o PySpark no ambiente Colab

In [2]:
import pyspark #Importa a biblioteca PySpark.
from pyspark.sql import SparkSession #Importa a classe SparkSession, que é o ponto de entrada para todas as funcionalidades do PySpark quando se usa DataFrames
from pyspark.sql.types import * #Importa todos os tipos de dados do PySpark SQL (como StringType, IntegerType), úteis para definir o esquema de um DataFrame
import datetime #Importa o módulo datetime para usar datetime.
from datetime import date #Importa o módulo date do Python

In [3]:
spark = SparkSession.builder.getOrCreate() ##Cria ou obtém uma instância do SparkSession.
                                           #O builder constrói a sessão, e getOrCreate() garante que uma sessão Spark esteja disponível para uso.
                                           #Este objeto (spark) é usado para ler dados.

Leitura Básica de Dados e Inspeção

In [4]:
df = spark.read.csv('dados_clinica_ficticios.csv') #Lê o arquivo CSV e cria um DataFrame (df)

In [5]:
df.take(2) #Retorna uma lista com as 5 primeiras linhas do DataFrame

[Row(_c0='Cod_Atendimento', _c1='Paciente', _c2='Data_Atendimento', _c3='Hora_Atendimento', _c4='Data_Hora', _c5='Convenio', _c6='Atendimento', _c7='Especialidade', _c8='Origem', _c9='Profissional', _c10='Procedimentos', _c11='Reagendamento', _c12='Teleconsulta', _c13='TCM_Plano', _c14='TCM_Unidade', _c15='Idade', _c16='Sexo'),
 Row(_c0='ATD1000', _c1='Fernanda Santos', _c2='2024-08-31', _c3='15:45', _c4='2024-08-31 15:45', _c5='SulAmérica', _c6='Consulta', _c7='Dermatologia', _c8='Encaminhamento', _c9='Dra. Lima', _c10='Raio-X', _c11='Sim', _c12='Sim', _c13='Plano B', _c14='Unidade Norte', _c15='89', _c16='Masculino')]

In [6]:
df.show(5) #Exibe as 5 primeiras linhas do DataFrame em um formato de tabela legível no console.

+---------------+-----------------+----------------+----------------+----------------+--------------+-----------+-------------+--------------+------------+----------------+-------------+------------+---------+-------------+-----+---------+
|            _c0|              _c1|             _c2|             _c3|             _c4|           _c5|        _c6|          _c7|           _c8|         _c9|            _c10|         _c11|        _c12|     _c13|         _c14| _c15|     _c16|
+---------------+-----------------+----------------+----------------+----------------+--------------+-----------+-------------+--------------+------------+----------------+-------------+------------+---------+-------------+-----+---------+
|Cod_Atendimento|         Paciente|Data_Atendimento|Hora_Atendimento|       Data_Hora|      Convenio|Atendimento|Especialidade|        Origem|Profissional|   Procedimentos|Reagendamento|Teleconsulta|TCM_Plano|  TCM_Unidade|Idade|     Sexo|
|        ATD1000|  Fernanda Santos|     

In [7]:
df.printSchema() #Exibe o esquema do DataFrame (nomes das colunas e seus tipos de dados).

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)



Leitura Avançada com Opções

In [8]:
df = spark.read.option('header', 'true').csv('dados_clinica_ficticios.csv') #Recarrega o CSV, mas desta vez instrui o Spark a usar a primeira linha do arquivo como nomes das colunas (header, true)
                                                                            #As colunas ainda serão do tipo StringType.

In [9]:
df = spark.read.option('header','true').option('inferSchema', 'true').csv('dados_clinica_ficticios.csv') #Recarrega o CSV, agora com o cabeçalho (header) e com a opção inferSchema como true
                                                                                                        #Isso faz com que o Spark analise os dados para tentar adivinhar (inferir) o tipo de dado correto de cada coluna

In [10]:
df = (spark.read        #Define explicitamente o separador (vírgula), usa o cabeçalho, descarta linhas malformadas (DROPMALFORMED) e infere os tipos de dados
      .option('sep',',')
      .option('header','true')
      .option('mode','DROPMALFORMED')
      .option('inferSchema' , 'true')
      .csv('dados_clinica_ficticios.csv'))

Definição Manual de Esquema

In [11]:
lista_campos = [                  #Inicia a definição do esquema. É uma lista de objetos StructField, onde cada um descreve uma coluna.
    StructField('Cod_Atendimento', StringType()),
    StructField('Paciente', StringType()),
    StructField('Data_Atendimento',DateType()),
    StructField('Hora_Atendimento', TimestampType()),
    StructField('Data_Hora', TimestampType()),
    StructField('Convenio', StringType()),
    StructField('Atendimento', StringType()),
    StructField('Especialidade', StringType()),
    StructField('Origem', StringType()),
    StructField('Profissional', StringType()),
    StructField('Procedimentos', StringType()),
    StructField('Reagendamento', StringType()),
    StructField('Teleconsulta', StringType()),
    StructField('TCM_Plano', StringType()),
    StructField('TCM_Unidade',StringType()),
    StructField('Idade', IntegerType()),
    StructField('Sexo', StringType())
]
schema_definido = StructType(lista_campos) #Cria o objeto final do Esquema (StructType) usando a lista de definições de campos.

In [12]:

df = spark.read.option('header', 'true').schema(schema_definido).csv('dados_clinica_ficticios.csv') #Lê o CSV com cabeçalho (header) e aplica o esquema definido manualmente (schema(schema_definido))
                                                                                                    #O Spark agora usará esses tipos de dados sem precisar inferir.

Criação de DataFrame a Partir de Dados em Memória

In [13]:
#criação de dados
dados_teste = [     #Define os dados em memória. É uma lista de objetos Row do PySpark, onde cada Row representa uma linha do DataFrame.
    Row(Cod_Atendimento='ATD1000', Paciente='Fernanda Santos', Data_Atendimento=datetime.date(2024, 8, 31), Hora_Atendimento=datetime.datetime(2025, 11, 13, 15, 45), Data_Hora=datetime.datetime(2024, 8, 31, 15, 45), Convenio='SulAmérica', Atendimento='Consulta', Especialidade='Dermatologia', Origem='Encaminhamento', Profissional='Dra. Lima', Procedimentos='Raio-X', Reagendamento='Sim', Teleconsulta='Sim', TCM_Plano='Plano B', TCM_Unidade='Unidade Norte', Idade=89, Sexo='Masculino'),
    Row(Cod_Atendimento='ATD1001', Paciente='Marcos Silva', Data_Atendimento=datetime.date(2024, 10, 28), Hora_Atendimento=datetime.datetime(2025, 11, 13, 9, 0), Data_Hora=datetime.datetime(2024, 10, 28, 9, 0), Convenio='Particular', Atendimento='Avaliação', Especialidade='Neurologia', Origem='Espontâneo', Profissional='Dr. Martins', Procedimentos='Consulta Clínica', Reagendamento='Não', Teleconsulta='Sim', TCM_Plano='Plano C', TCM_Unidade='Unidade Sul', Idade=70, Sexo='Masculino')
]

In [14]:
df_parcial = spark.createDataFrame(data=dados_teste, schema=schema_definido)

#df_parcial = spark.createDataFrame(data=dados_teste)

In [15]:
df_parcial.printSchema() #Exibe o esquema do DataFrame recém-criado, confirmando que os tipos de dados definidos (DateType, TimestampType, etc.) foram aplicados corretamente.

root
 |-- Cod_Atendimento: string (nullable = true)
 |-- Paciente: string (nullable = true)
 |-- Data_Atendimento: date (nullable = true)
 |-- Hora_Atendimento: timestamp (nullable = true)
 |-- Data_Hora: timestamp (nullable = true)
 |-- Convenio: string (nullable = true)
 |-- Atendimento: string (nullable = true)
 |-- Especialidade: string (nullable = true)
 |-- Origem: string (nullable = true)
 |-- Profissional: string (nullable = true)
 |-- Procedimentos: string (nullable = true)
 |-- Reagendamento: string (nullable = true)
 |-- Teleconsulta: string (nullable = true)
 |-- TCM_Plano: string (nullable = true)
 |-- TCM_Unidade: string (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Sexo: string (nullable = true)



In [16]:
df_parcial.show() #Exibe todo o conteúdo do pequeno DataFrame criado em memória.

+---------------+---------------+----------------+-------------------+-------------------+----------+-----------+-------------+--------------+------------+----------------+-------------+------------+---------+-------------+-----+---------+
|Cod_Atendimento|       Paciente|Data_Atendimento|   Hora_Atendimento|          Data_Hora|  Convenio|Atendimento|Especialidade|        Origem|Profissional|   Procedimentos|Reagendamento|Teleconsulta|TCM_Plano|  TCM_Unidade|Idade|     Sexo|
+---------------+---------------+----------------+-------------------+-------------------+----------+-----------+-------------+--------------+------------+----------------+-------------+------------+---------+-------------+-----+---------+
|        ATD1000|Fernanda Santos|      2024-08-31|2025-11-13 15:45:00|2024-08-31 15:45:00|SulAmérica|   Consulta| Dermatologia|Encaminhamento|   Dra. Lima|          Raio-X|          Sim|         Sim|  Plano B|Unidade Norte|   89|Masculino|
|        ATD1001|   Marcos Silva|      2